<a href="https://colab.research.google.com/github/adithisirpa/Automatic-Attendance-Marker-using-face-recognition-/blob/main/Automatic_Attendance_Marker_Using_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install face_recognition 
! pip install Xlsxwriter

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=76b78e10f36a29d778f3663c0c4faa9cf29661766589a2a7e62e7d239974f014
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models
     |████████████████████████████████| 153kB 8.5MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab.patches import cv2_imshow 
import cv2
import os
import face_recognition
import imutils 

In [ ]:
# this reads the image and resizes it 
def reads_img (path):
  image = cv2.imread(path)
  ratio = image.shape[0] /500.0
  actual_img = image.copy()
  image = imutils.resize(image, height = 500)
  return image

In [ ]:
# this creates the encodings for registered student directory and the attended students directory and compares their faces 
registeredppl_encodings = []
registeredppl_names = []
registeredppl_dir = '/content/drive/MyDrive/registeredppl'

present = []
for file in os.listdir(registeredppl_dir):
  img = reads_img(registeredppl_dir + '/' + file)
  img_encodings = face_recognition.face_encodings(img)[0]
  registeredppl_encodings.append(img_encodings)
  registeredppl_names.append(file.split('.')[0])

ppl_dir = '/content/drive/MyDrive/ppl'
for file in os.listdir(ppl_dir):
  #print ("the choosen file is ",file)
  img = reads_img(ppl_dir + '/' + file)
  img_encodings = face_recognition.face_encodings(img)[0]
  comparedresults = face_recognition.compare_faces(registeredppl_encodings,img_encodings)
  for a in range(len(comparedresults)):
    if comparedresults[a]:
      present.append(registeredppl_names[a])
#print(present)

In [ ]:
# this creates absent list
ppl_dir = '/content/drive/MyDrive/ppl'
ppl_names =[]
absent = []
for file in os.listdir(ppl_dir):
  img_ppl = reads_img(ppl_dir + '/' + file)
  ppl_names.append(file.split('.')[0])

for a in range(len(ppl_names)):
  if ppl_names[a] not in present:
    absent.append(ppl_names[a])
#print(absent)

In [ ]:
# this to mark the attendance for the above conclusions 
import xlsxwriter
from xlrd import open_workbook
from xlwt import Workbook

workbook = xlsxwriter.Workbook("Attendance Sheet.xlsx")
worksheet = workbook.add_worksheet()

bold = workbook.add_format({'bold': True})

worksheet.set_column(0, 10, 25)

row = 0
column = 0
worksheet.write(row,column,'Names of Students',bold)
worksheet.write(row,column+1,'Attendance Status',bold)

row = row+1

#this prints the student list and student status
for i in range(len(present)):
  worksheet.write(row,column,present[i])
  worksheet.write(row,column+1,'Present')
  row = row+1

for i in range(len(absent)):
  worksheet.write(row,column,absent[i])
  worksheet.write(row,column+1,'Absent')
  row = row+1

workbook.close()
